In [1]:
import os
import numpy as np
import pandas as pd
import random
import seaborn as sns

import datetime as datetime
import matplotlib.dates as dates
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from contextlib import contextmanager
from time import time
from tqdm import tqdm
import lightgbm as lgbm

from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

In [2]:
data0 = pd.read_csv("../input/covid19-indonesia/covid_19_indonesia_time_series_all.csv")
data0[-5:]

,Date,Location ISO Code,Location,New Cases,New Deaths,New Recovered,New Active Cases,Total Cases,Total Deaths,Total Recovered,...,Latitude,New Cases per Million,Total Cases per Million,New Deaths per Million,Total Deaths per Million,Total Deaths per 100rb,Case Fatality Rate,Case Recovered Rate,Growth Factor of New Cases,Growth Factor of New Deaths
31817,9/15/2022,ID-SA,Sulawesi Utara,37,0,0,37,52770,1213,50997,...,1.259638,14.01,19974.38,0.0,459.14,45.91,2.30%,96.64%,2.85,1.00
31818,9/15/2022,ID-SB,Sumatera Barat,13,0,3,10,104640,2371,102066,...,-0.850253,2.36,18959.11,0.0,429.59,42.96,2.27%,97.54%,6.50,1.00
31819,9/15/2022,ID-SS,Sumatera Selatan,16,0,1,15,82198,3376,78510,...,-3.216212,1.95,10002.74,0.0,410.83,41.08,4.11%,95.51%,3.20,1.00
31820,9/15/2022,ID-SU,Sumatera Utara,50,0,5,45,158866,3288,154924,...,2.191894,3.36,10680.15,0.0,221.04,22.10,2.07%,97.52%,1.92,1.00
31821,9/16/2022,IDN,Indonesia,2358,27,2997,-666,6405044,157876,6218708,...,-0.789275,8.89,24153.07,0.1,595.34,59.53,2.46%,97.09%,0.89,1.29


In [3]:
data0['Location'].unique()

array(['DKI Jakarta', 'Indonesia', 'Riau', 'Jawa Barat', 'Banten',
       'Jawa Tengah', 'Sulawesi Tenggara', 'Bali', 'Kalimantan Timur',
       'Daerah Istimewa Yogyakarta', 'Sumatera Utara', 'Jawa Timur',
       'Kepulauan Riau', 'Sulawesi Selatan', 'Jambi', 'Maluku', 'Papua',
       'Maluku Utara', 'Sumatera Selatan', 'Aceh', 'Kalimantan Tengah',
       'Lampung', 'Sulawesi Tengah', 'Sulawesi Utara', 'Sumatera Barat',
       'Papua Barat', 'Kalimantan Utara', 'Sulawesi Barat',
       'Kalimantan Barat', 'Kalimantan Selatan',
       'Kepulauan Bangka Belitung', 'Bengkulu', 'Nusa Tenggara Barat',
       'Nusa Tenggara Timur', 'Gorontalo'], dtype=object)

In [4]:
import datetime
from datetime import date
date2=[]
for item in data0['Date']:
    item2=item.split('/')
    month=int(item2[0])
    day=int(item2[1])
    year=int(item2[2])
    date2+=[datetime.date(year,month,day)]
data0['Date2']=date2
data0

,Date,Location ISO Code,Location,New Cases,New Deaths,New Recovered,New Active Cases,Total Cases,Total Deaths,Total Recovered,...,New Cases per Million,Total Cases per Million,New Deaths per Million,Total Deaths per Million,Total Deaths per 100rb,Case Fatality Rate,Case Recovered Rate,Growth Factor of New Cases,Growth Factor of New Deaths,Date2
0,3/1/2020,ID-JK,DKI Jakarta,2,0,0,2,39,20,75,...,0.18,3.60,0.0,1.84,0.18,51.28%,192.31%,NaN,NaN,2020-03-01
1,3/2/2020,ID-JK,DKI Jakarta,2,0,0,2,41,20,75,...,0.18,3.78,0.0,1.84,0.18,48.78%,182.93%,1.00,1.00,2020-03-02
2,3/2/2020,IDN,Indonesia,2,0,0,2,2,0,0,...,0.01,0.01,0.0,0.00,0.00,0.00%,0.00%,NaN,NaN,2020-03-02
3,3/2/2020,ID-RI,Riau,1,0,0,1,1,0,1,...,0.16,0.16,0.0,0.00,0.00,0.00%,100.00%,NaN,NaN,2020-03-02
4,3/3/2020,ID-JK,DKI Jakarta,2,0,0,2,43,20,75,...,0.18,3.96,0.0,1.84,0.18,46.51%,174.42%,1.00,1.00,2020-03-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31817,9/15/2022,ID-SA,Sulawesi Utara,37,0,0,37,52770,1213,50997,...,14.01,19974.38,0.0,459.14,45.91,2.30%,96.64%,2.85,1.00,2022-09-15
31818,9/15/2022,ID-SB,Sumatera Barat,13,0,3,10,104640,2371,102066,...,2.36,18959.11,0.0,429.59,42.96,2.27%,97.54%,6.50,1.00,2022-09-15
31819,9/15/2022,ID-SS,Sumatera Selatan,16,0,1,15,82198,3376,78510,...,1.95,10002.74,0.0,410.83,41.08,4.11%,95.51%,3.20,1.00,2022-09-15
31820,9/15/2022,ID-SU,Sumatera Utara,50,0,5,45,158866,3288,154924,...,3.36,10680.15,0.0,221.04,22.10,2.07%,97.52%,1.92,1.00,2022-09-15


In [5]:
data1=data0.groupby('Date2',as_index=False).sum()
data1

,Date2,New Cases,New Deaths,New Recovered,New Active Cases,Total Cases,Total Deaths,Total Recovered,Total Active Cases,City or Regency,...,Population Density,Longitude,Latitude,New Cases per Million,Total Cases per Million,New Deaths per Million,Total Deaths per Million,Total Deaths per 100rb,Growth Factor of New Cases,Growth Factor of New Deaths
0,2020-03-01,2,0,0,2,39,20,75,-56,0.0,...,16334.31,106.836118,-6.204699,0.18,3.60,0.00,1.84,0.18,0.00,0.00
1,2020-03-02,5,0,0,5,44,20,76,-52,0.0,...,16542.45,322.562555,-6.482326,0.35,3.95,0.00,1.84,0.18,1.00,1.00
2,2020-03-03,3,1,0,2,47,21,136,-110,0.0,...,17819.00,430.166263,-13.402758,0.20,4.15,0.02,1.86,0.18,1.00,3.00
3,2020-03-04,3,0,0,3,50,21,136,-107,0.0,...,17819.00,430.166263,-13.402758,0.20,4.36,0.00,1.86,0.18,4.00,3.00
4,2020-03-05,1,1,0,0,51,22,136,-107,0.0,...,17819.00,430.166263,-13.402758,0.02,4.39,0.09,1.96,0.19,3.00,3.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
925,2022-09-12,1995,30,6991,-5026,12786964,315669,12408267,63028,0.0,...,25440.02,3980.507699,-95.127843,20.79,894765.12,1.34,20383.66,2038.36,8.88,31.14
926,2022-09-13,3087,24,4209,-1146,12790051,315693,12412476,61882,0.0,...,25440.02,3980.507699,-95.127843,30.72,894795.81,0.78,20384.44,2038.43,49.39,29.43
927,2022-09-14,4699,27,4301,371,12794750,315720,12416777,62253,0.0,...,25440.02,3980.507699,-95.127843,194.02,894989.85,0.29,20384.74,2038.46,159.98,32.05
928,2022-09-15,5293,22,4159,1112,12770491,314861,12392302,63328,0.0,...,25242.21,3738.786622,-93.353099,265.25,873438.00,0.32,19720.01,1971.97,100.15,29.00


In [6]:
data2=data1[['Date2','New Cases']]
data2

,Date2,New Cases
0,2020-03-01,2
1,2020-03-02,5
2,2020-03-03,3
3,2020-03-04,3
4,2020-03-05,1
...,...,...
925,2022-09-12,1995
926,2022-09-13,3087
927,2022-09-14,4699
928,2022-09-15,5293


In [7]:
data2['ratio to 7 days before']=1

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [8]:
data2

,Date2,New Cases,ratio to 7 days before
0,2020-03-01,2,1
1,2020-03-02,5,1
2,2020-03-03,3,1
3,2020-03-04,3,1
4,2020-03-05,1,1
...,...,...,...
925,2022-09-12,1995,1
926,2022-09-13,3087,1
927,2022-09-14,4699,1
928,2022-09-15,5293,1


In [9]:
n=len(data2)
for i in range(n):    
    cpi=data2['New Cases'][i]
    data2.loc[i+7,'positives 7 days before']=cpi
data2[100:114]

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http

,Date2,New Cases,ratio to 7 days before,positives 7 days before
100,2020-06-09,2079.0,1.0,1208.0
101,2020-06-10,2517.0,1.0,1389.0
102,2020-06-11,1938.0,1.0,1226.0
103,2020-06-12,2145.0,1.0,1428.0
104,2020-06-13,2126.0,1.0,1830.0
105,2020-06-14,1733.0,1.0,1539.0
106,2020-06-15,2133.0,1.0,1721.0
107,2020-06-16,2172.0,1.0,2079.0
108,2020-06-17,2083.0,1.0,2517.0
109,2020-06-18,2634.0,1.0,1938.0


In [10]:
data2['ratio to 7 days before']=data2['New Cases']*100/data2['positives 7 days before']
data2['mean ratio to 7 days before']=data2['ratio to 7 days before'].rolling(window=7).mean()
data3=data2[431:-7]
data3

,Date2,New Cases,ratio to 7 days before,positives 7 days before,mean ratio to 7 days before
431,2021-05-06,11332.0,96.879542,11697.0,95.599204
432,2021-05-07,12644.0,115.491414,10948.0,97.688543
433,2021-05-08,12311.0,135.943021,9056.0,103.068229
434,2021-05-09,7869.0,89.167139,8825.0,101.781313
435,2021-05-10,9734.0,104.017953,9358.0,105.072510
...,...,...,...,...,...
925,2022-09-12,1995.0,42.628205,4680.0,70.737373
926,2022-09-13,3087.0,42.791794,7214.0,66.687060
927,2022-09-14,4699.0,66.880159,7026.0,65.242965
928,2022-09-15,5293.0,84.337157,6276.0,66.341309


In [11]:
fig=make_subplots(specs=[[{"secondary_y":False}]])
fig.add_trace(go.Scatter(x=data3['Date2'],y=data3['ratio to 7 days before'],name='ratio to 7 days before'),secondary_y=False,)
fig.add_trace(go.Scatter(x=data3['Date2'],y=data3['mean ratio to 7 days before'],name='mean ratio to 7 days before'),secondary_y=False,)
fig.update_layout(autosize=False,width=700,height=500,title_text="Ratio to 7 days before in Indonesia")
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Ratio",secondary_y=False)
fig.show()

In [12]:
data4=data3[-7:]
data4

,Date2,New Cases,ratio to 7 days before,positives 7 days before,mean ratio to 7 days before
923,2022-09-10,5218.0,80.042951,6519.0,78.499706
924,2022-09-11,3878.0,70.164646,5527.0,76.293187
925,2022-09-12,1995.0,42.628205,4680.0,70.737373
926,2022-09-13,3087.0,42.791794,7214.0,66.687060
927,2022-09-14,4699.0,66.880159,7026.0,65.242965
928,2022-09-15,5293.0,84.337157,6276.0,66.341309
929,2022-09-16,2358.0,42.047076,5608.0,61.270284


In [13]:
N=[]
for i in range(40):
    N+=[i]
NUMB=pd.DataFrame(N)

In [14]:
data5=pd.concat([data4,NUMB],axis=0)
data5=data5.drop(0,axis=1).reset_index(drop=True)
data5[0:10]

,Date2,New Cases,mean ratio to 7 days before,positives 7 days before,ratio to 7 days before
0,2022-09-10,5218.0,78.499706,6519.0,80.042951
1,2022-09-11,3878.0,76.293187,5527.0,70.164646
2,2022-09-12,1995.0,70.737373,4680.0,42.628205
3,2022-09-13,3087.0,66.687060,7214.0,42.791794
4,2022-09-14,4699.0,65.242965,7026.0,66.880159
5,2022-09-15,5293.0,66.341309,6276.0,84.337157
6,2022-09-16,2358.0,61.270284,5608.0,42.047076
7,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN


In [15]:
data5['pcr_positives150']=data5['New Cases']
data5['pcr_positives130']=data5['New Cases']
data5['pcr_positives90']=data5['New Cases']

In [16]:
for i in range(7,47):
    data5.loc[i,'Date2']=data5.loc[i-1,'Date2']+datetime.timedelta(days=1)
    data5.loc[i,'pcr_positives150']=data5.loc[i-7,'pcr_positives150']*1.5
    data5.loc[i,'pcr_positives130']=data5.loc[i-7,'pcr_positives130']*1.3    
    data5.loc[i,'pcr_positives90']=data5.loc[i-7,'pcr_positives90']*0.9
data6=data5[['Date2','New Cases','pcr_positives150','pcr_positives130','pcr_positives90']]
data6

,Date2,New Cases,pcr_positives150,pcr_positives130,pcr_positives90
0,2022-09-10,5218.0,5218.000000,5218.000000,5218.000000
1,2022-09-11,3878.0,3878.000000,3878.000000,3878.000000
2,2022-09-12,1995.0,1995.000000,1995.000000,1995.000000
3,2022-09-13,3087.0,3087.000000,3087.000000,3087.000000
4,2022-09-14,4699.0,4699.000000,4699.000000,4699.000000
5,2022-09-15,5293.0,5293.000000,5293.000000,5293.000000
6,2022-09-16,2358.0,2358.000000,2358.000000,2358.000000
7,2022-09-17,NaN,7827.000000,6783.400000,4696.200000
8,2022-09-18,NaN,5817.000000,5041.400000,3490.200000
9,2022-09-19,NaN,2992.500000,2593.500000,1795.500000


In [17]:
fig=make_subplots(specs=[[{"secondary_y":False}]])
fig.add_trace(go.Scatter(x=data6['Date2'],y=data6['pcr_positives150'],name='150% incresase to 7-days before'),secondary_y=False,)
fig.add_trace(go.Scatter(x=data6['Date2'],y=data6['pcr_positives130'],name='130% incresase to 7-days before'),secondary_y=False,)
fig.add_trace(go.Scatter(x=data6['Date2'],y=data6['pcr_positives90'],name='90% incresase to 7-days before'),secondary_y=False,)
fig.add_trace(go.Scatter(x=data6['Date2'],y=data6['New Cases'],name='actual daily new cases'),secondary_y=False,)
fig.update_layout(autosize=False,width=700,height=500,title_text="Prediction of daily new cases in Indonesia as of 2021/07/09")
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Number",secondary_y=False)
fig.show()